In [1]:
from google.colab import drive
import tensorflow as tf
import pickle

# Mount Google Drive
drive.mount('/content/drive')

# Load model
model_load_path = "/content/drive/MyDrive/cnn_speaker_classifier.h5"
model = tf.keras.models.load_model(model_load_path)

print("Model loaded successfully!")

# Load label encoder
le_load_path = "/content/drive/MyDrive/label_encoder.pkl"

with open(le_load_path, 'rb') as f:
    le = pickle.load(f)

print("Label Encoder loaded successfully!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Model loaded successfully!
Label Encoder loaded successfully!


In [2]:
import tensorflow as tf
from tensorflow.keras import layers, Model, backend as K
import numpy as np
import librosa
import pickle
import os
import random

In [3]:
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D, Dense, BatchNormalization, Lambda
from tensorflow.keras.optimizers import Adam

# Contrastive Loss Function
def contrastive_loss(y_true, y_pred):
    margin = 1.0
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

# Feature Extractor Model
def build_feature_extractor():
    inp = Input(shape=(128, 128, 1))
    x = Conv2D(32, (3,3), activation='relu', padding='same')(inp)
    x = MaxPooling2D(pool_size=(2,2))(x)

    x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)  # Improved generalization
    x = Dropout(0.3)(x)
    x = MaxPooling2D(pool_size=(2,2))(x)

    x = Conv2D(128, (3,3), activation='relu', padding='same')(x)  # Extra Conv Layer
    x = GlobalAveragePooling2D()(x)  # Replaces Flatten

    model = Model(inputs=inp, outputs=x)

    # Unfreeze the last convolutional block for fine-tuning
    for layer in model.layers[-3:]:
        layer.trainable = True

    return model

# Siamese Network
def build_siamese_network():
    input_a = Input(shape=(128, 128, 1))
    input_b = Input(shape=(128, 128, 1))

    feature_extractor = build_feature_extractor()

    feat_a = feature_extractor(input_a)
    feat_b = feature_extractor(input_b)

    # Euclidean Distance Calculation
    distance = Lambda(lambda tensors: K.sqrt(K.sum(K.square(tensors[0] - tensors[1]), axis=-1, keepdims=True)))([feat_a, feat_b])

    model = Model(inputs=[input_a, input_b], outputs=distance)

    return model

# Build the model
siamese_model = build_siamese_network()
siamese_model.compile(loss=contrastive_loss, optimizer=Adam(learning_rate=0.0005))

siamese_model.summary()


Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 128, 128, 1)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, 128, 128, 1)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional_8 (Functional) │ (None, 128)            │         92,928 │ input_layer[0][0],     │
│                           │                        │                │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda (Lambda)           │ (None, 1)              │              0 │ functional_8[0][0],    │
│                           │                        │                │ functional_8[1][0]     │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 92,928 (363.00 KB)

 Trainable params: 92,800 (362.50 KB)

 Non-trainable params: 128 (512.00 B)

In [4]:
import os
import random

def create_siamese_pairs(data_dir):
    # Get the list of speaker directories (i.e., id1001, id1002, etc.)
    speakers = [s for s in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, s))]
    speaker_files = {}

    for speaker in speakers:
        speaker_path = os.path.join(data_dir, speaker)
        sessions = os.listdir(speaker_path)  # Get sessions inside the speaker's directory
        speaker_files[speaker] = []

        for session in sessions:
            session_path = os.path.join(speaker_path, session)
            if os.path.isdir(session_path):
                files = [f for f in os.listdir(session_path) if os.path.isfile(os.path.join(session_path, f))]
                full_paths = [os.path.join(session_path, f) for f in files]
                speaker_files[speaker].extend(full_paths)  # Store full file paths for the speaker

    positive_pairs = []
    negative_pairs = []
    labels = []

    # Create positive pairs (same speaker, different recordings)
    for speaker, files in speaker_files.items():
        if len(files) > 1:  # Ensure there are at least two recordings
            for i in range(len(files) - 1):
                positive_pairs.append((files[i], files[i+1]))
                labels.append(1)

    # Create negative pairs (different speakers)
    speaker_list = list(speaker_files.keys())
    for _ in range(len(positive_pairs)):  # Ensure equal number of negative pairs
        speaker1, speaker2 = random.sample(speaker_list, 2)  # Pick two different speakers
        if speaker_files[speaker1] and speaker_files[speaker2]:  # Ensure both have files
            file1 = random.choice(speaker_files[speaker1])
            file2 = random.choice(speaker_files[speaker2])
            negative_pairs.append((file1, file2))
            labels.append(0)

    # Combine positive and negative pairs
    all_pairs = positive_pairs + negative_pairs
    print(f"Total pairs created: {len(all_pairs)}")  #Debugging: Check total number of pairs
    return all_pairs, labels

data_dir = "/content/drive/MyDrive/Datasets/vox_indian/"
pairs, labels = create_siamese_pairs(data_dir)

# Debugging: Check the result
print(f"Number of pairs: {len(pairs)}")
print(f"Number of labels: {len(labels)}")
print("\nSample Pairs:")
for i in range(5):  # Print first 5 pairs
    print(pairs[i], "Label:", labels[i])


Total pairs created: 9382
Number of pairs: 9382
Number of labels: 9382

Sample Pairs:
('/content/drive/MyDrive/Datasets/vox_indian/id10002/6WO410QOeuo/00010.wav', '/content/drive/MyDrive/Datasets/vox_indian/id10002/6WO410QOeuo/00004.wav') Label: 1
('/content/drive/MyDrive/Datasets/vox_indian/id10002/6WO410QOeuo/00004.wav', '/content/drive/MyDrive/Datasets/vox_indian/id10002/6WO410QOeuo/00011.wav') Label: 1
('/content/drive/MyDrive/Datasets/vox_indian/id10002/6WO410QOeuo/00011.wav', '/content/drive/MyDrive/Datasets/vox_indian/id10002/6WO410QOeuo/00003.wav') Label: 1
('/content/drive/MyDrive/Datasets/vox_indian/id10002/6WO410QOeuo/00003.wav', '/content/drive/MyDrive/Datasets/vox_indian/id10002/6WO410QOeuo/00008.wav') Label: 1
('/content/drive/MyDrive/Datasets/vox_indian/id10002/6WO410QOeuo/00008.wav', '/content/drive/MyDrive/Datasets/vox_indian/id10002/6WO410QOeuo/00007.wav') Label: 1


In [5]:
import traceback

def preprocess_audio(file_path, target_shape=(128, 128), sr=16000):
    try:
        if not os.path.exists(file_path):
            print(f"File not found: {file_path}")
            return None

        print(f"Processing: {file_path}")

        y, _ = librosa.load(file_path, sr=sr)
        print(f"Loaded audio shape: {y.shape}")
        y = librosa.util.fix_length(y, size=3 * sr)

        mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=target_shape[0], hop_length=512)
        mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
        print(f"Mel spectrogram shape: {mel_spec_db.shape}")

        mel_spec_db = (mel_spec_db - mel_spec_db.min()) / (mel_spec_db.max() - mel_spec_db.min() + 1e-6)    #min max normalization here (0 <-> 1)
        mel_spec_db = tf.image.resize(mel_spec_db[None, :,:, None], target_shape[:2]).numpy().squeeze()     #resizing and remove unnecessary
        print(f"Resized shape: {mel_spec_db.shape}")

        mel_spec_db = mel_spec_db[:,:, np.newaxis]    #adds channel dimension , so basically converting our 2d spectogram into a 3d tensor for our cnn to understand

        print("Processing successful!\n")
        return np.expand_dims(mel_spec_db, axis=0) #Reshape to (batch_size,128,128,1) as it is accepting a 4d tensor

    except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")
        traceback.print_exc()  # Print full error traceback
        return None

In [6]:
def pad_spectrogram(spectrogram, target_shape=(128, 128)):
    rows, cols = spectrogram.shape[:2]
    pad_height = max(0, target_shape[0] - rows) #height for making 128
    pad_width = max(0, target_shape[1] - cols)  #Widht for making 128. overall padded to 128 x 128 if necessary

    padded_spectrogram = np.pad(spectrogram, ((0, pad_height), (0, pad_width), (0, 0)), mode='constant')
    return padded_spectrogram

In [ ]:
import os
dataset_path = "/content/drive/MyDrive/Datasets/vox_indian/"
for speaker_id in os.listdir(dataset_path):
    speaker_path = os.path.join(dataset_path, speaker_id)
    if os.path.isdir(speaker_path):
        print(f"Speaker {speaker_id}: {os.listdir(speaker_path)}")

Speaker id10002: ['6WO410QOeuo', 'eNc4LrrvV80', '0_laIeN-Q44', 'gaQqIoV_aLY', 'cMGEuZ1zqXk', 'C7k7C-PDvAA', 'TqUbiOgEb0w', 'w9H-ZMvdE9M', 'RLKKsYiCMvc', 'wd_7oYV4dsU', 'xTV-jFAUKcw', 'VMaXdHLz5Bk', 'SI4D2_YXvBE', 'QnnjJ9i5WFs', 'Mpr9wqUuLQA', 'QanuGhOhb9A', 'Y2Gr1I2DO7M']
Speaker id10724: ['6tFEkfsIeXk', '9pNGll4IBls', 'DP5gCFoWgGI', 'R8Pw9GUJQ08', 'T37JG3wNeTo', 'JNMeEXwXbqw', 'fUKPz7jr6NE', 'gqGgO5EQ0HQ', 'uw8SDDK6Mbk', 'pKOCOOG5Ukw', 'kHSikgTOz70', 'e-vuV1-N97Q', 'YsVDo2AO_X0', 'XVoRDzwHwDU', 'xS_Mh-DOAI0']
Speaker id10583: ['05gr63nmJS8', 'p7FgxxM6Aw8', 'BceLH6tkbH0', 'iqOLK7gLNCo', 'NwhpR7Mc5FY', 'eLstqbGd0rU', 'ncajg8Tki3M', '8iZJ5S_OZqQ', 'AfPv4Jhk2NQ', 'qMMlFEm6CuE', 'HywWPDQKSYE', 'Worxr8OXuQI', 'R3iDRYV7SpU', 'U0UNvoRitTg', 'XeF78f1ApJM', 'rrPb_EgwUD0', 'wm-zfibgpBo', 'SvGvgDL-Hp4', 'WuZqoSEQKOE']
Speaker id10324: ['4BD7f4QwiYc', '1BN1Twr0pDM', '4ZjZnQ1sITc', '49dNaFf6l-4', 'B1nRjIlyftI', 'jx1tdxlQpB0', 'fTl25XI6GN0', 'jFe5HnJWW_s', 'gQXwYYPcN7Y', 'DE089Obo6L4', 'DNqi4NOsLO0'

In [7]:
# Before processing, filter out invalid pairs
valid_pairs = []
valid_labels = []

for file1, file2 in pairs:
    if os.path.exists(file1) and os.path.exists(file2):
        valid_pairs.append((file1, file2))
    else:
        print(f"Skipping pair: {file1} | {file2}")

print(f"Found {len(valid_pairs)} valid pairs out of {len(pairs)}")

Found 9382 valid pairs out of 9382


In [7]:
valid_pairs = [(file1, file2) for file1, file2 in pairs if all(map(os.path.exists, (file1, file2)))]  # modified code for faster exec

In [8]:
X1 = []
X2 = []
y_cleaned = []

for i, (file1, file2) in enumerate(pairs):
    if not os.path.exists(file1):
        print(f"File not found: {file1}")
        continue
    if not os.path.exists(file2):
        print(f"File not found: {file2}")
        continue
    x1 = preprocess_audio(file1)
    x2 = preprocess_audio(file2)
    if x1 is not None and x2 is not None:
        x1 = pad_spectrogram(x1[0])
        x2 = pad_spectrogram(x2[0])

        X1.append(x1)
        X2.append(x2)
        y_cleaned.append(labels[i])

X1 = np.array(X1)
X2 = np.array(X2)
y = np.array(y_cleaned)


# Reshape X1 and X2 before fitting
X1 = X1.reshape(-1, 128, 128, 1)  # Reshape to (num_samples, 128, 128, 1)
X2 = X2.reshape(-1, 128, 128, 1)  # Reshape to (num_samples, 128, 128, 1)
print(f"Processed {len(X1)} valid pairs out of {len(pairs)} total pairs.")


Streaming output truncated to the last 5000 lines.

Processing: /content/drive/MyDrive/Datasets/vox_indian/id10724/xS_Mh-DOAI0/00002.wav
Loaded audio shape: (315521,)
Mel spectrogram shape: (128, 94)
Resized shape: (128, 128)
Processing successful!

Processing: /content/drive/MyDrive/Datasets/vox_indian/id10583/HywWPDQKSYE/00015.wav
Loaded audio shape: (109441,)
Mel spectrogram shape: (128, 94)
Resized shape: (128, 128)
Processing successful!

Processing: /content/drive/MyDrive/Datasets/vox_indian/id10956/h_vFlQ6YUAU/00003.wav
Loaded audio shape: (103041,)
Mel spectrogram shape: (128, 94)
Resized shape: (128, 128)
Processing successful!

Processing: /content/drive/MyDrive/Datasets/vox_indian/id10393/1D22Tgx1k4A/00012.wav
Loaded audio shape: (90241,)
Mel spectrogram shape: (128, 94)
Resized shape: (128, 128)
Processing successful!

Processing: /content/drive/MyDrive/Datasets/vox_indian/id11089/dg1HrPpOCHE/00014.wav
Loaded audio shape: (95361,)
Mel spectrogram shape: (128, 94)
Resized sh

In [ ]:
# X1 = np.array([preprocess_audio(p[0]) for p in pairs])
# X2 = np.array([preprocess_audio(p[1]) for p in pairs])
# y = np.array(labels)

# # Remove None values (failed audio files)
# valid_idx = [i for i in range(len(X1)) if X1[i] is not None and X2[i] is not None]
# X1 = np.array([X1[i] for i in valid_idx])
# X2 = np.array([X2[i] for i in valid_idx])
# y = np.array([y[i] for i in valid_idx])

# Step 1: Preprocess and filter valid pairs in a single pass
processed_data = [
    (x1, x2, label)
    for (file1, file2), label in zip(pairs, labels)
    if (x1 := preprocess_audio(file1)) is not None and (x2 := preprocess_audio(file2)) is not None
]

# Step 2: Unpack into separate arrays
X1, X2, y = map(np.array, zip(*processed_data)) if processed_data else (np.array([]), np.array([]), np.array([]))

print(f"Processed {len(X1)} valid pairs out of {len(pairs)} total pairs.")


Processing: /content/drive/MyDrive/Datasets/vox_indian/id10002/6WO410QOeuo/00010.wav
Loaded audio shape: (124801,)
Mel spectrogram shape: (128, 94)
Resized shape: (128, 128)
Processing successful!

Processing: /content/drive/MyDrive/Datasets/vox_indian/id10002/6WO410QOeuo/00004.wav
Loaded audio shape: (104321,)
Mel spectrogram shape: (128, 94)
Resized shape: (128, 128)
Processing successful!

Processing: /content/drive/MyDrive/Datasets/vox_indian/id10002/6WO410QOeuo/00004.wav
Loaded audio shape: (104321,)
Mel spectrogram shape: (128, 94)
Resized shape: (128, 128)
Processing successful!

Processing: /content/drive/MyDrive/Datasets/vox_indian/id10002/6WO410QOeuo/00011.wav
Loaded audio shape: (80641,)
Mel spectrogram shape: (128, 94)
Resized shape: (128, 128)
Processing successful!

Processing: /content/drive/MyDrive/Datasets/vox_indian/id10002/6WO410QOeuo/00011.wav
Loaded audio shape: (80641,)
Mel spectrogram shape: (128, 94)
Resized shape: (128, 128)
Processing successful!

Processing: 

KeyboardInterrupt: 

In [ ]:
X1 = np.squeeze(X1, axis=1)
X2 = np.squeeze(X2, axis=1)
y = y.reshape(-1, 1)

print(f"X1 shape: {X1.shape}, X2 shape: {X2.shape}, y shape: {y.shape}")

ValueError: cannot select an axis to squeeze out which has size not equal to one

In [10]:
import numpy as np
import tensorflow as tf

if len(X1) == 0 or len(X2) == 0 or len(y) == 0:
    raise ValueError(" One or more datasets are empty")

# Convert to float32
X1 = np.array(X1, dtype=np.float32)
X2 = np.array(X2, dtype=np.float32)
y = np.array(y, dtype=np.float32).reshape(-1, 1)

# Convert to tensors
X1_tensor = tf.convert_to_tensor(X1, dtype=tf.float32)
X2_tensor = tf.convert_to_tensor(X2, dtype=tf.float32)
y_tensor = tf.convert_to_tensor(y, dtype=tf.float32)
print(siamese_model.input_shape)


[(None, 128, 128, 1), (None, 128, 128, 1)]


In [ ]:
from sklearn.model_selection import train_test_split

X1_train, X1_temp, X2_train, X2_temp, y_train, y_temp = train_test_split(X1, X2, y, test_size=0.3, random_state=42)
X1_val, X1_test, X2_val, X2_test, y_val, y_test = train_test_split(X1_temp, X2_temp, y_temp, test_size=1/3, random_state=42)

# Convert to tensors
X1_train_tensor = tf.convert_to_tensor(X1_train, dtype=tf.float32)
X2_train_tensor = tf.convert_to_tensor(X2_train, dtype=tf.float32)
y_train_tensor = tf.convert_to_tensor(y_train, dtype=tf.float32)

X1_val_tensor = tf.convert_to_tensor(X1_val, dtype=tf.float32)
X2_val_tensor = tf.convert_to_tensor(X2_val, dtype=tf.float32)
y_val_tensor = tf.convert_to_tensor(y_val, dtype=tf.float32)

X1_test_tensor = tf.convert_to_tensor(X1_test, dtype=tf.float32)
X2_test_tensor = tf.convert_to_tensor(X2_test, dtype=tf.float32)
y_test_tensor = tf.convert_to_tensor(y_test, dtype=tf.float32)

# Train model with explicit validation set
siamese_model.fit(
    [X1_train_tensor, X2_train_tensor], y_train_tensor,
    batch_size=32, epochs=10,
    validation_data=([X1_val_tensor, X2_val_tensor], y_val_tensor)
)

# Evaluate on test set
loss, accuracy = siamese_model.evaluate([X1_test_tensor, X2_test_tensor], y_test_tensor)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

Epoch 1/10
206/206 ━━━━━━━━━━━━━━━━━━━━ 568s 3s/step - loss: 0.1707 - val_loss: 0.2839
Epoch 2/10
206/206 ━━━━━━━━━━━━━━━━━━━━ 621s 3s/step - loss: 0.1221 - val_loss: 0.1580
Epoch 3/10
206/206 ━━━━━━━━━━━━━━━━━━━━ 621s 3s/step - loss: 0.1190 - val_loss: 0.1290
Epoch 4/10
206/206 ━━━━━━━━━━━━━━━━━━━━ 559s 3s/step - loss: 0.1129 - val_loss: 0.1258
Epoch 5/10
206/206 ━━━━━━━━━━━━━━━━━━━━ 564s 3s/step - loss: 0.1105 - val_loss: 0.1341
Epoch 6/10
206/206 ━━━━━━━━━━━━━━━━━━━━ 557s 3s/step - loss: 0.1061 - val_loss: 0.1274
Epoch 7/10
206/206 ━━━━━━━━━━━━━━━━━━━━ 567s 3s/step - loss: 0.1038 - val_loss: 0.1247
Epoch 8/10
206/206 ━━━━━━━━━━━━━━━━━━━━ 561s 3s/step - loss: 0.1019 - val_loss: 0.1220
Epoch 9/10
173/206 ━━━━━━━━━━━━━━━━━━━━ 1:23 3s/step - loss: 0.1016

In [1]:
model.save("siamese_model_contrastive.h5")

NameError: name 'model' is not defined

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate_siamese_model(model, test_pairs, test_labels):

    predictions = model.predict(test_pairs)
    predicted_labels = (predictions > 0.5).astype(int)  # Convert to binary

    # Compute evaluation metrics
    acc = accuracy_score(test_labels, predicted_labels)
    precision = precision_score(test_labels, predicted_labels)
    recall = recall_score(test_labels, predicted_labels)
    f1 = f1_score(test_labels, predicted_labels)

    print(f"Model Evaluation Metrics:")
    print(f"Accuracy: {acc:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-score: {f1:.4f}")

    # Get training history
    history = model.history.history  # Access training logs

    # Plot Accuracy and Loss Curves
    plt.figure(figsize=(12, 5))

    # Loss Plot
    plt.subplot(1, 2, 1)
    plt.plot(history.get('loss', []), label='Train Loss') # Use .get() with default value
    plt.plot(history.get('val_loss', []), label='Val Loss') # Use .get() with default value
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Loss Curve')
    plt.legend()

    # Accuracy Plot
    plt.subplot(1, 2, 2)
    plt.plot(history.get('accuracy', []), label='Train Accuracy') # Use .get() with default value
    plt.plot(history.get('val_accuracy', []), label='Val Accuracy') # Use .get() with default value
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Accuracy Curve')
    plt.legend()

    plt.show()

In [ ]:
def predict_similarity(file1, file2):
    preprocessed_1 = preprocess_audio(file1)  # Shape (1, 128, 128, 1)
    preprocessed_2 = preprocess_audio(file2)  # Shape (1, 128, 128, 1)

    if preprocessed_1 is None or preprocessed_2 is None:
        print("Error processing files")
        return None

    preprocessed_1 = pad_spectrogram(preprocessed_1[0])
    preprocessed_2 = pad_spectrogram(preprocessed_2[0])

    # Reshape to (1, 128, 128, 1) to match the model's input shape
    preprocessed_1 = preprocessed_1.reshape(1, 128, 128, 1)
    preprocessed_2 = preprocessed_2.reshape(1, 128, 128, 1)

    similarity = siamese_model.predict([preprocessed_1, preprocessed_2])
    return similarity[0][0]

test_file1 = "/content/drive/MyDrive/Datasets/vox_indian/id10017/7QOGR0zvTxY/00002.wav"
test_file2 = "/content/drive/MyDrive/Datasets/vox_indian/id10912/6wBjQ6soleU/00003.wav"

score = predict_similarity(test_file1, test_file2)
print(f"Similarity Score: {score:.2f}") # Corrected variable name here
if score < 1.5:
    print("similar")
else:
    print("different")

Processing: /content/drive/MyDrive/Datasets/vox_indian/id10017/7QOGR0zvTxY/00002.wav
Loaded audio shape: (110081,)
Mel spectrogram shape: (128, 94)
Resized shape: (128, 128)
Processing successful!

Processing: /content/drive/MyDrive/Datasets/vox_indian/id10912/6wBjQ6soleU/00003.wav
Loaded audio shape: (136321,)
Mel spectrogram shape: (128, 94)
Resized shape: (128, 128)
Processing successful!

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
Similarity Score: -5.33
different


In [ ]:
evaluate_siamese_model(siamese_model, (X1_test_tensor, X2_test_tensor), y_test_tensor)

30/30 ━━━━━━━━━━━━━━━━━━━━ 122s 4s/step


ValueError: Input arrays use different devices: /job:localhost/replica:0/task:0/device:CPU:0, cpu